In [1]:
from ImageProcessor import Frame
from deuces import Card
from deuces import Evaluator
import time
import os

In [2]:
frame = Frame(blinds = .05)

In [18]:
playOn = True
ai = AI()

while playOn:
    
    frame.update()
 
    while frame.identify_card('hole1') is 'None' or frame.identify_card('hole2') is 'None':
        print 'no cards found'
        time.sleep(3)
        frame.update()
        
    c1 = frame.identify_card('hole1')
    c2 = frame.identify_card('hole2') 
    
    Card.print_pretty_cards([Card.new(c1), Card.new(c2)])
    #print c1, c2
    ## check to see if we should fold the cards immediately
    if frame.user_position() != 'BB' and ai.initial_screening(c1, c2) is False:          
        print 'FOLDING'
        frame.fold()
        time.sleep(.5)
        frame.fold()
        time.sleep(8)
        
        
    else:    
        ##wait until user turn  
        time.sleep(3)
        frame.update()
        
        #print 'postion' , frame.user_position()
        print 'is turn? : ' , frame.is_user_turn()


        while(not frame.is_user_turn()):
            time.sleep(3)
            frame.update()
            #print frame.user_position()
    
        
        ## BOT MAKES MOVE
        bet = ai.calculate_bet(frame, c1, c2)
        
        print bet
        
        if bet == 'fold':
            frame.fold()
            time.sleep(4)
            continue
        elif bet == 'call':
            frame.call()
            
            
            
        else:
            frame.bet(bet)
         
        #print 'BOT MAKES DECISION HERE'
        os.system("say 'Your turn'")
        
        
        break
    
    
    

no cards found
no cards found
  [ 3 ♣ ] , [ 7 ♠ ]  
FOLDING
no cards found
no cards found
  [ 5 ♠ ] , [ Q ❤ ]  
FOLDING
no cards found
no cards found
  [ A ♣ ] , [ 9 ♠ ]  
is turn? :  False
fold
no cards found
  [ 5 ❤ ] , [ 9 ♠ ]  
FOLDING
  [ T ♠ ] , [ 2 ♦ ]  
FOLDING
no cards found
  [ 7 ♣ ] , [ 2 ❤ ]  
FOLDING
no cards found
no cards found
  [ T ♣ ] , [ A ♦ ]  
FOLDING
  [ 2 ♠ ] , [ 4 ♣ ]  
is turn? :  False
fold
no cards found
  [ A ♦ ] , [ 9 ♦ ]  
is turn? :  False
fold


KeyboardInterrupt: 

In [14]:
class AI():
    
    def __init__(self):
        '''If our hand is not in this list, fold IMMEDIATELY (unless big blind)'''
        self.pairs_1 = ['AA', 'KK', 'QQ']
        self.pairs_2 = ['JJ', 'TT', '99'] 
        self.pairs_3 = ['88', '77']
        self.pairs_4 = ['66', '55', '44', '33', '22']
        self.pairs = self.pairs_1 + self.pairs_2 + self.pairs_3 + self.pairs_4
        
        self.suited_connectors_1 = ['KQs', 'QJs', 'JTs']
        self.suited_connectors_2 = ['9Ts', '89s', '78s', '67s']
        self.suited_connectors_3 = ['56s', '45s', '34s', '23s']
        self.suited_connectors = self.suited_connectors_1 + self.suited_connectors_2 + self.suited_connectors_3
            
        self.suited_connectors_one_gap_1 = ['KJs', 'QTs', 'J9s']
        self.suited_connectors_one_gap_2 = ['8Ts', '79s', '68s']
        self.suited_connectors_one_gap_3 = ['57s', '46s', '35s', '24s']
        self.suited_connectors_one_gap = self.suited_connectors_one_gap_1 + self.suited_connectors_one_gap_2 + self.suited_connectors_one_gap_3
        
        self.suited_connectors_two_gap_1 = ['KTs', 'Q9s', 'J8s']
        self.suited_connectors_two_gap_2 = ['7Ts', '69s', '58s']
        self.suited_connectors_two_gap_3 = ['47s', '36s', '25s']
        self.suited_connectors_two_gap = self.suited_connectors_two_gap_1 + self.suited_connectors_two_gap_2 + self.suited_connectors_two_gap_3
        
        self.suited_aces_1 = ['AJs', 'ATs']
        self.suited_aces_2 = ['6As', '7As', '8As', '9As']
        self.suited_aces_3 = ['2As', '3As', '4As', '5As']
        self.suited_aces = self.suited_aces_1 + self.suited_aces_2 + self.suited_aces_3 
        
        self.big_aces = ['AKs', 'AK', 'AQs', 'AQs']
        
        
        

        self.play_list = self.pairs + self.suited_connectors + self.suited_connectors_one_gap + \
                         self.suited_connectors_two_gap + self.suited_aces  + self.big_aces
                   
                    

                    
        ################################################################
        #                       UTG
        ################################################################
        '''If we are under the gun, raise by 2x if hand in this list'''    
        self.UO_UTG_2x = self.pairs_4 + self.suited_connectors_1 + self.suited_connectors_2 \
                      + self.suited_connectors_one_gap_2 + self.suited_aces_3 + self.suited_aces_1
                        

        '''If we are under the gun, raise by 3x if hand in this list'''
        self.UO_UTG_3x = self.pairs_1 + self.pairs_2 + self.pairs_3 + self.big_aces
        
        
        self.UTG = {'unopened' : [(self.UO_UTG_3x, 3), (self.UO_UTG_2x, 2)]}        
        
        
        ################################################################
        #                       MP
        ################################################################
        
           
        '''If we are Unopened, raise by 2x if hand in this list'''
        self.UO_MP_2x = self.pairs_4 + self.suited_connectors_2 + self.suited_connectors_one_gap \
                     + self.suited_aces_3
        
        self.UO_MP_3x = self.pairs_1 + self.pairs_2 + self.pairs_3 + self.suited_connectors_1 + self.suited_connectors_2\
                     + self.suited_aces_1 + self.big_aces
            
        
        
        '''If only one limper'''
        self.OC_MP_6x = self.pairs_1
        self.OC_MP_5x = ['AKs', 'JJ']
        self.OC_MP_4x = ['TT', '99', 'AK', 'AQs']
        self.OC_MP_3x = self.pairs_3 + self.suited_aces_1
        self.OC_MP_call = self.pairs_4 + self.suited_connectors_1 + self.suited_connectors_2\
                            + self.suited_connectors_one_gap_1 + self.suited_connectors_one_gap_2\
                            + self.suited_aces_3
                
        OC = [( self.OC_MP_6x, 6), ( self.OC_MP_5x, 5), (self.OC_MP_4x, 4), (self.OC_MP_3x, 3), 
                   (self.OC_MP_call, 'call')]
                
                
        
        
        self.MP = {'unopened' : [(self.UO_MP_3x, 3), (self.UO_MP_2x, 2)], 'limpers':[OC]}     
        
        
        
        ################################################################
        #                       CO
        ################################################################
        
        self.UO_CO_3x = self.pairs + self.suited_connectors_1 + self.suited_connectors_2 + self.suited_connectors_one_gap_1\
                        + self.suited_connectors_one_gap_2 + self.suited_aces_1 + self.suited_connectors_3 + self.big_aces
        
                  
        'If only one limper'
        self.OC_CO_6x = self.pairs_1
        self.OC_CO_5x = ['AKs', 'JJ']
        self.OC_CO_4x = ['TT', '99', 'AK', 'AQs']
        self.OC_CO_3x = self.pairs_3 + self.suited_aces_1
        self.OC_CO_call = self.pairs_4 + self.suited_connectors_1 + self.suited_connectors_2\
                            + self.suited_connectors_one_gap_1 + self.suited_connectors_one_gap_2\
                            + self.suited_aces_3
                
        OC = [( self.OC_CO_6x, 6), ( self.OC_CO_5x, 5), (self.OC_CO_4x, 4), (self.OC_CO_3x, 3), 
                   (self.OC_CO_call, 'call')]
                
                
        'If two limpers'
        self.TC_CO_8x = self.pairs_1
        self.TC_CO_6x = self.pairs_2 + self.big_aces
        self.TC_CO_call = self.pairs_3 + self.pairs_4 + self.suited_connectors + self.suited_connectors_one_gap\
                            + self.suited_connectors_two_gap + self.suited_aces
            
        TC = [( self.TC_CO_8x, 8), ( self.TC_CO_6x, 6), (self.TC_CO_call, 'call')]
            
            
        
        self.CO = {'unopened' : [(self.UO_CO_3x, 3)], 'limpers':[OC, TC]} 
        
        
        
        ################################################################
        #                       BTN
        ################################################################
        self.UO_BTN_3x = self.pairs + self.suited_connectors + self.suited_connectors_one_gap\
                        + self.suited_connectors_two_gap + self.suited_aces + self.big_aces
            
        'If only one limper'
        self.OC_BTN_5x = self.pairs_1
        self.OC_BTN_4x = self.pairs_2 + self.big_aces
        self.OC_BTN_3x = self.pairs_3 + self.pairs_4 + self.suited_connectors_1 + self.suited_connectors_2\
                         + self.suited_connectors_one_gap_1 + self.suited_connectors_one_gap_2\
                         + self.suited_aces_1
        self.OC_BTN_call = self.suited_connectors_3 + self.suited_connectors_one_gap_3 + self.suited_connectors_two_gap\
                           + self.suited_aces_2 + self.suited_aces_3
            
        OC = [( self.OC_BTN_5x, 5), ( self.OC_BTN_4x, 4), (self.OC_BTN_4x, 3), (self.OC_BTN_call, 'call')]
                
                
        'If two limpers'
        self.TC_BTN_7x = self.pairs_1
        self.TC_BTN_5x = self.pairs_2 + self.big_aces
        self.TC_BTN_call = self.pairs_3 + self.pairs_4 + self.suited_connectors + self.suited_connectors_one_gap\
                            + self.suited_connectors_two_gap + self.suited_aces
            
        TC = [( self.TC_BTN_7x, 7), ( self.TC_BTN_5x, 5), (self.TC_BTN_call, 'call')]
            
        
        'If three limpers'
        self.ThC_BTN_10x = self.pairs_1
        self.ThC_BTN_6x = self.pairs_2 + self.big_aces
        self.ThC_BTN_call = self.pairs_3 + self.pairs_4 + self.suited_connectors + self.suited_connectors_one_gap\
                            + self.suited_connectors_two_gap + self.suited_aces
            
        ThC = [( self.ThC_BTN_10x, 10), ( self.ThC_BTN_6x, 6), (self.ThC_BTN_call, 'call')]

        
        
        
        self.BTN = {'unopened' : [(self.UO_BTN_3x, 3)], 'limpers':[OC, TC, ThC]} 
        
        ################################################################
        #                       SB
        ################################################################
        self.UO_SB_3x = self.pairs + self.suited_connectors_1 + self.suited_connectors_2 \
                        + self.suited_connectors_one_gap_1 +  self.suited_connectors_one_gap_2\
                        + self.suited_connectors_two_gap_1  + self.suited_aces + self.big_aces
        
        
        'If only one limper'
        self.OC_SB_7x = self.pairs_1
        self.OC_SB_call = self.pairs_2 + self.pairs_3 + self.pairs_4\
                            + self.suited_connectors + self.suited_connectors_one_gap + self.suited_connectors_two_gap\
                            + self.big_aces + self.suited_aces
        OC = [(self.OC_SB_7x, 7), (self.OC_SB_call, 'call')]
                
                
        'If two limpers'
        self.TC_SB_10x = self.pairs_1
        self.TC_SB_call = self.pairs_2 + self.pairs_3 + self.pairs_4\
                            + self.suited_connectors + self.suited_connectors_one_gap + self.suited_connectors_two_gap\
                            + self.big_aces + self.suited_aces
        TC = [(self.TC_SB_10x, 10), (self.TC_SB_call, 'call')]
            
        
        'If three limpers'
        self.ThC_SB_15x = self.pairs_1
        self.ThC_SB_call = self.pairs_2 + self.pairs_3 + self.pairs_4\
                            + self.suited_connectors + self.suited_connectors_one_gap + self.suited_connectors_two_gap\
                            + self.big_aces + self.suited_aces
        ThC = [(self.ThC_SB_15x, 15), (self.ThC_SB_call, 'call')]
                
        'If four limpers'
        self.FC_SB_20x = self.pairs_1
        self.FC_SB_call = self.pairs_2 + self.pairs_3 + self.pairs_4\
                            + self.suited_connectors + self.suited_connectors_one_gap + self.suited_connectors_two_gap\
                            + self.big_aces + self.suited_aces
                
        FC = [(self.FC_SB_20x, 20), (self.FC_SB_call, 'call')]
        
        self.SB = {'unopened' : [(self.UO_SB_3x, 3)], 'limpers':[OC, TC, ThC, FC]}
       
        ################################################################
        #                       BB
        ################################################################
        
        'If only one limper'
        self.OC_BB_7x = self.pairs_1
        self.OC_BB_call = self.pairs_2 + self.pairs_3 + self.pairs_4\
                            + self.suited_connectors + self.suited_connectors_one_gap + self.suited_connectors_two_gap\
                            + self.big_aces + self.suited_aces
        OC = [(self.OC_BB_7x, 7), (self.OC_BB_call, 'call')]
                
                
        'If two limpers'
        self.TC_BB_10x = self.pairs_1
        self.TC_BB_call = self.pairs_2 + self.pairs_3 + self.pairs_4\
                            + self.suited_connectors + self.suited_connectors_one_gap + self.suited_connectors_two_gap\
                            + self.big_aces + self.suited_aces
        TC = [(self.TC_BB_10x, 10), (self.TC_BB_call, 'call')]
            
        
        'If three limpers'
        self.ThC_BB_15x = self.pairs_1
        self.ThC_BB_call = self.pairs_2 + self.pairs_3 + self.pairs_4\
                            + self.suited_connectors + self.suited_connectors_one_gap + self.suited_connectors_two_gap\
                            + self.big_aces + self.suited_aces
        ThC = [(self.ThC_BB_15x, 15), (self.ThC_BB_call, 'call')]
                
        'If four limpers'
        self.FC_BB_20x = self.pairs_1
        self.FC_BB_call = self.pairs_2 + self.pairs_3 + self.pairs_4\
                            + self.suited_connectors + self.suited_connectors_one_gap + self.suited_connectors_two_gap\
                            + self.big_aces + self.suited_aces
        FC = [(self.FC_BB_20x, 20), (self.FC_BB_call, 'call')]
                
        'If five limpers'
        self.FiC_BB_25x = self.pairs_1
        self.FiC_BB_call = self.pairs_2 + self.pairs_3 + self.pairs_4\
                            + self.suited_connectors + self.suited_connectors_one_gap + self.suited_connectors_two_gap\
                            + self.big_aces + self.suited_aces
                
        FiC = [(self.FiC_BB_25x, 25), (self.FiC_BB_call, 'call')]
        
        
        self.BB = {'limpers':[OC, TC, ThC, FC, FiC]}
        
        
        
        ############################################################
        
  
        
        self.master = {'UTG' : self.UTG, 'MP' : self.MP, 'CO' : self.CO, 'BTN' : self.BTN, 'SB' : self.SB, 'BB' : self.BB}
    
    
    
    
    
    
    
    def encode_hand(self, c1, c2):
        if c1[1] == c2[1]:
            encoded_hand = c1[0]+c2[0]+'s'
        else:
            encoded_hand = c1[0]+c2[0] 
        return ''.join(sorted(encoded_hand))
    
    def initial_screening(self, c1, c2):
        '''Returns false if we should fold hand immediately'''
        hand = self.encode_hand(c1, c2)
        return hand in self.play_list
    
    def calculate_bet(self, frame, c1, c2):
        
        hand = self.encode_hand(c1, c2)
        bets = frame.bets()
        blinds = frame.blinds
        position = frame.user_position()
        
        #check if hand is unopened
        #print blinds
        #print bets
        
        #print 'number of callers / raisers' , len(bets[bets >= blinds])    
        table = self.master[position]
        
        
        #True if unopened to us
        if position != 'BB' and len(bets[bets >= blinds]) == 1:
            print "Unopened"
            table = table['unopened']    
            for raise_amount_list in table:
                if hand in raise_amount_list[0]:
                        return raise_amount_list[1] * blinds
                    
        #True if no raisers
        elif bets.max() == blinds:
            print "Limpers!"
            table = table['limpers']
            num_callers = len(bets[bets == blinds]) - 1
            
            if position is 'BB':
                num_callers += 1
            
            table = table[num_callers - 1]
                
            for raise_amount_list in table:
                if hand in raise_amount_list[0]:
                        return raise_amount_list[1] * blinds
                
            
        
      
        return 'fold'
                
        
        
    
    
    
    
    
    
    